<a href="https://colab.research.google.com/github/wowthecoder/Image-Classification-Tutorial/blob/main/Facial_emotion_recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Setup and Load Data

### 1.1 Installing Dependencies and Setup

In [ ]:
!pip install tensorflow opencv-python matplotlib


---


In [1]:
import tensorflow as tf
import os

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
gpus

In [3]:
#Avoid OOM(Out of Memory) errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

### 1.2 Load Data

In [4]:
import numpy as np
from matplotlib import pyplot as plt

train_ds = tf.keras.utils.image_dataset_from_directory(
    '../input/fer2013/train', 
    batch_size=64, 
    image_size=(48, 48), 
    color_mode='grayscale', 
    validation_split=0.2,
    subset="training",
    seed=123,
)

In [5]:
val_ds = tf.keras.utils.image_dataset_from_directory(
    '../input/fer2013/train', 
    batch_size=64, 
    image_size=(48, 48), 
    color_mode='grayscale', 
    validation_split=0.2,
    subset="validation",
    seed=123,
)

The code below shows that in 1 batch, there are 64 images of size 48x48px, and each image has 1 color channel(grayscale)

In [6]:
train_iterator = train_ds.as_numpy_iterator()
# Get another batch from the iterator
batch = train_iterator.next()

In [7]:
# Images represented as numpy arrays
batch[0].shape

There are 7 classes so the labels are numbered 0-6.

In [8]:
# The image labels
batch[1]

Check which number is assigned to which label.

In [10]:
fig, ax = plt.subplots(ncols=6, figsize=(20, 20))
for idx, img in enumerate(batch[0][:6]):
    ax[idx].imshow(img.squeeze())
    ax[idx].title.set_text(batch[1][idx])


*   0: angry
*   1: disgust
*   2: fear
*   3: happy
*   4: neutral
*   5: sad
*   6: surprise









# 2. Preprocess Data

### 2.1 Scale Data and One-hot encoding

 Scale our data values from 0-255(RGB values) to 0-1   
 One-hot encoding is for Categorical entropy

In [6]:
train_ds = train_ds.map(lambda x,y: (x/255, tf.one_hot(y, depth=7)))
val_ds = val_ds.map(lambda x,y: (x/255, tf.one_hot(y, depth=7)))

The code below is for testing.

In [7]:
scaled_iterator = train_ds.as_numpy_iterator()
batch = scaled_iterator.next()
batch[0].max()

### 2.2 Optimise Performance

In [8]:
AUTOTUNE = tf.data.AUTOTUNE   

train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)   
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# 3. Deep Learning

### 3.1 Build Deep Learning Model

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from keras.regularizers import l2

model = Sequential([
    Conv2D(filters=64, kernel_size=(3,3), strides=1, activation='relu', input_shape=(48,48,1)),
    Conv2D(filters=64, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2), strides=(2,2)),
    Dropout(0.5),

    Conv2D(filters=128, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2D(filters=128, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2D(filters=128, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.5),

    Conv2D(filters=256, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2D(filters=256, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2D(filters=256, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.5),

    Conv2D(filters=512, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2D(filters=512, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
    BatchNormalization(),

    Conv2D(filters=512, kernel_size=(3,3), strides=1, padding='same', activation='relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2)),
    Dropout(0.5),

    Flatten(),
    Dense(units=512, activation='relu'),
    Dropout(0.5),
    Dense(units=256, activation='relu'),
    Dropout(0.5),
    Dense(units=128, activation='relu'),
    Dropout(0.5),
    Dense(units=64, activation='relu'),
    Dropout(0.5),
    # softmax for multi-class classification
    Dense(units=7, activation='softmax'),
])

In [13]:
adam = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    optimizer=adam,
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [14]:
model.summary()

### 3.2 Train Model

In [15]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint

In [16]:
logdir='facial_emotions/logs'

Callback functions:
1.   **ReduceLROnPlateau**
> This function monitors the validation loss for signs of a plateau and then modify the learning rate by the specified factor when a plateau is detected.





In [17]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)

2.   **EarlyStopping**

> The function waits for a number of epochs and terminates the training if no change in the parameter is found.





In [18]:
early_stopping = EarlyStopping(
    monitor='val_acc',
    min_delta=0,
    patience=6,
    mode='auto',
)

3.   **ModelCheckpoint**

> Saves the model after every epoch. In case training stops, we can load the checkpoint and resume the progress.









In [19]:
checkpointer = ModelCheckpoint(
    filepath=logdir,
    monitor='val_loss',
    verbose=1,
    save_best_only=True,
)

In [20]:
hist = model.fit(
    train_ds,
    epochs=100,
    validation_data=val_ds,
    shuffle=True,
    callbacks=[lr_reducer, early_stopping, checkpointer],
)

Highest validation accuracy achieved: **61.19%**    

### 3.3 Plot performance

In [22]:
plt.style.use('seaborn-whitegrid')
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper right")
plt.show()

In [23]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='acc')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_acc')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 4. Evaluate Performance

### 4.1 Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, Accuracy

TP = True Positive, TN = True Negative, FP = False Positive, FN = False Negative
1. Accuracy   
>  *(Number of correct predictions) / (Total number of predictions)*    
2. Precision   
>  What proportion of positive identifications was actually correct?   
   *(TP) / (TP + FP)* 
3. Recall   
>  What proportion of actual positives was identified correctly?   
   *(TP) / (TP + FN)*

In [ ]:
from sklearn.metrics import 

### 4.2 Test (with test dataset)

In [24]:
test_data = tf.keras.utils.image_dataset_from_directory(
    '../input/fer2013/test',
    batch_size=64,
    image_size=(48,48),
    color_mode='grayscale',
)   
test_data = test_data.map(lambda x,y: (x/255, tf.one_hot(y, depth=7)))

In [25]:
predictions = model.predict(test_data)

In [28]:
scores = model.evaluate(test_data)
print(model.metrics_names)
print(scores)

Highest accuracy achieved: **60.73%**    
Benchmarks: 
* human-level accuracy - 65±5%    
* highest performing published work - 76.82%

### 4.3 Test (with random images)

# 5. Save the Model